# lolapi.py Usage

Below is a demonstration of how to use the lolapi.py wrapper to retreive match data from the Riot Games API.

In [1]:
import lolapi as la

Input `apikey` and `summonername`.

API key is assigned through the [Riot Developer Portal](https://developer.riotgames.com/).

Summoner name is the player's name as it appears in game (capitalization must match).

In [20]:
apikey = ''
summonername = ''

Encrypted account ID is retrieved using the `retrieve_accountid()` function:

In [21]:
accountid = la.retrieve_accountid(apikey, summonername)

Pass the `accountid` to collect matches for that account.

Several filter parameters are available to use:
`champion` - ID of champion
`queue` - Queue type ID
`endtime` - End time of match in epoch seconds
`begintime` - Begin time of match in epoch seconds
`limit` - Limit number of matches to retrieve
`server` - Server ID (e.g. 'na1' for North America)

In this example, queue ID 450 is used to filter for the 'ARAM' game mode.

If collecting over 1000 matches the request may receive a timeout error. If this happens the program will wait 2 minutes, then continue with the request. Riot API only allows for 100 requests every 2 minutes.

In [4]:
account_matches_df = la.retrieve_account_matches(apikey, accountid, limit=100, queue=450)
print(str(len(account_matches_df)) + ' matches retrieved.')

100 matches retrieved.


In [5]:
for column in account_matches_df.columns:
    print(column)

champion
gameId
lane
platformId
queue
role
season
timestamp


Use the 'gameId' column from the retrieved matches as an input for the `retrieve_match_data()` function. This will bring in the detailed data for each match. Due to the request volume restriction the program will pause for 2 mins when it receives a timeout or resource exceeded response. If it receives a second error after waiting it will cancel.

Use the `limit` parameter to select only a specific number of matches. These are collected in the same order as the list of match IDs given. If given limit is over the total possible matches it will collect all matches.

The server parameter is available in this function, though it should match the server ID used in `retrieve_account_matches()`.

In [6]:
match_detail_data = la.retrieve_match_data(apikey, account_matches_df['gameId'], limit=5)

5 matches collected.


The collected match data can be parsed using the `parse_match_data()` function. The match data is parsed into 3 separate dataframes:
`match_data_df` - Contains overall match data
`player_data_df` - Contains info and metrics for individual players
`team_data_df` - Contains info and metrics for each team

The resulting dataframes can be connected via the `gameId` column as a primary key. Each `gameId` will have 2 teams with IDs of '100' and '200', and 10 players with player IDs from 1 to 10. These dataframes can be used in a relational database.

In [7]:
match_data_df, player_data_df, team_data_df = la.parse_match_data(match_detail_data)

match_data_df.head(1)

,creation,duration,gameid,mapid,mode,platformid,queueid,seasonid,type,version
0,1598575166222,971,3552641376,12,ARAM,NA1,450,13,MATCHED_GAME,10.16.330.9186


In [8]:
player_data_df.head(1)

,assists,champLevel,championid,combatPlayerScore,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,doubleKills,firstBloodAssist,...,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,unrealKills,visionScore,visionWardsBoughtInGame,win
0,22,16,42,0,461,461,5593,4,0,False,...,4,0,0,0,602,0,0,0,0,True


In [9]:
team_data_df.head(1)

,bans,baronKills,dominionVictoryScore,dragonKills,firstBaron,firstBlood,firstDragon,firstInhibitor,firstRiftHerald,firstTower,gameid,inhibitorKills,riftHeraldKills,teamId,towerKills,vilemawKills,win
0,[],0,0,0,False,False,False,True,False,False,3552641376,1,0,100,4,0,Win


Champion data is another valuable piece of information and can be retrieved using the `retrieve_champ_info()` function. This collects overall data for the champion not related to specific match performance and outputs as a dataframe. Champion info can be tied to players using the champion ID.

In [14]:
champ_df = la.retrieve_champ_info()
champ_df.head()

,attack,defense,difficulty,id,magic,name,role
0,8,4,4,266,3,Aatrox,Fighter
1,3,4,5,103,8,Ahri,Mage
2,5,3,7,84,8,Akali,Assassin
3,6,9,7,12,5,Alistar,Tank
4,2,6,3,32,8,Amumu,Tank


Similar functions are available for items and summoner spells using `retrieve_item_info()` and `retrieve_spell_info()`. Use the IDs to join item and spell info to players.

In [16]:
item_df = la.retrieve_item_info()
item_df.head(1)

,basecost,id,name,totalcost
0,300,1001,Boots of Speed,300


In [17]:
spell_df = la.retrieve_spell_info()
spell_df.head(1)

,id,name
0,21,Barrier
